# Transfer Learning


## imports and data processing:


In [5]:
# !gdown 11LQILlalZGi_skLIipnpTyh-At77B_j6 --output ../data/landmarks.zip
# !unzip ../data/landmarks.zip -d ../data/landmarks

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#os manipulation imports:
import os
import glob
import random


# tensorflow imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

plt.style.use('dark_background')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = sns.color_palette('dark'))
plt.rcParams['font.size'] = 15

In [ ]:
#visualize random images from each class from trainset:


In [21]:
datadir = '../data/landmarks/New_landmark_images'
traindataset = tf.keras.utils.image_dataset_from_directory(datadir + '/train', shuffle = True , seed = 123 , image_size = (227 , 227) , batch_size = 32)

validationdataset = tf.keras.utils.image_dataset_from_directory(datadir + '/validation' , shuffle = False , seed = 123 , image_size = (227 , 227) , batch_size= 32)

testdataset = tf.keras.utils.image_dataset_from_directory(datadir + '/test' , shuffle = False , seed = 123 , image_size = (227 , 227) , batch_size= 32)


Found 737 files belonging to 10 classes.
Found 155 files belonging to 10 classes.
Found 43 files belonging to 10 classes.
